In [1]:
#import system packages
import sys
import os
import warnings
#import scientific packages
import math
import pandas as pd
import numpy as np
import scipy as sc
from scipy.io import loadmat
import scipy.interpolate as sci
#import datetime
from datetime import date,datetime
import itertools
flatten = itertools.chain.from_iterable
#import own functions from utils.py
from utils import *
#import plotting tools
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline


In [7]:
# Define global data: with Python this all has to go in the same cell - alternative to container.map needs to be explored
DATA_DICTIONARY = [{'Country':'Bulgaria','keySet':'bg','valueSet':1,'datafile':'bulgaria_coronavirus.csv' , 'N':7*pow(10,6) , 'St-End_Date':[sdate2ordinal('14-Mar-2020'), sdate2ordinal('9-Apr-2020')], 'new_St_Date': sdate2ordinal('8-Mar-2020')},
                   {'Country':'China-ALL','keySet':'chi','valueSet':2,'datafile':'china_coronavirus.csv' ,  'N':11*pow(10,6), 'St-End_Date':[sdate2ordinal('5-Feb-2020'), sdate2ordinal('12-Feb-2020')], 'new_St_Date': sdate2ordinal('30-Jan-2020')},
                   {'Country':'China-Wuhan','keySet':'wuh','valueSet':3,'datafile':'china_hubei_wuhan_coronavirus.csv', 'N':11*pow(10,6), 'St-End_Date':[sdate2ordinal('5-Feb-2020'), sdate2ordinal('12-Feb-2020')], 'new_St_Date': sdate2ordinal('30-Jan-2020')},
                   {'Country':'Italy2','keySet':'it2','valueSet':4,'datafile':'italy_coronavirus__VER2.csv',  'N':60*pow(10,6),  'St-End_Date':[sdate2ordinal('1-Mar-2020'), sdate2ordinal('28-Mar-2020')], 'new_St_Date': sdate2ordinal('21-Feb-2020')},
                   {'Country':'Italy','keySet':'it','valueSet':5,'datafile':'italy_coronavirus.csv',  'N':60*pow(10,6),  'St-End_Date':[sdate2ordinal('1-Mar-2020'), sdate2ordinal('3-Apr-2020')], 'new_St_Date': sdate2ordinal('21-Feb-2020')},
                   {'Country':'Germany','keySet':'ger','valueSet':6,'datafile':'germany_coronavirus.csv',  'N':80*pow(10,6),  'St-End_Date':[sdate2ordinal('5-Mar-2020'), sdate2ordinal('8-Apr-2020')], 'new_St_Date': sdate2ordinal('25-Feb-2020')},
                   {'Country':'Spain','keySet':'sp','valueSet':7,'datafile':'spain_coronavirus.csv',  'N':47*pow(10,6),  'St-End_Date':[sdate2ordinal('5-Mar-2020'), sdate2ordinal('12-Mar-2020')], 'new_St_Date': sdate2ordinal('25-Feb-2020')},
                   {'Country':'Japan','keySet':'jap','valueSet':8,'datafile':'japan_coronavirus.csv',  'N':126*pow(10,6),  'St-End_Date':[sdate2ordinal('27-Feb-2020'), sdate2ordinal('16-Mar-2020')], 'new_St_Date': sdate2ordinal('14-Feb-2020')},
                   {'Country':'UK','keySet':'uk','valueSet':9,'datafile':'uk_coronavirus.csv',      'N':66*pow(10,6),  'St-End_Date':[sdate2ordinal('5-Mar-2020'), sdate2ordinal('3-Apr-2020')], 'new_St_Date': sdate2ordinal('25-Feb-2020')},
                   {'Country':'Germany2','keySet':'ger2','valueSet':10,'datafile':'germany_coronavirus_VER2.csv',  'N':80*pow(10,6),  'St-End_Date':[sdate2ordinal('5-Mar-2020'), sdate2ordinal('1-Apr-2020')], 'new_St_Date': sdate2ordinal('25-Feb-2020')},
                   {'Country':'Bulgaria2','keySet':'bg2','valueSet':11,'datafile':'bulgaria_coronavirus__VER2.csv',  'N':7*pow(10,6),  'St-End_Date':[sdate2ordinal('14-Mar-2020'), sdate2ordinal('4-Apr-2020')], 'new_St_Date': sdate2ordinal('8-Mar-2020')},
                   {'Country':'Russia','keySet':'rus','valueSet':12,'datafile':'russia_coronavirus.csv',  'N':150*pow(10,6),  'St-End_Date':[sdate2ordinal('15-Mar-2020'), sdate2ordinal('4-Apr-2020')], 'new_St_Date': sdate2ordinal('6-Mar-2020')},
                   {'Country':'US','keySet':'us','valueSet':13,'datafile':'us_coronavirus.csv',  'N':330*pow(10,6),  'St-End_Date':[sdate2ordinal('11-Mar-2020'), sdate2ordinal('31-Mar-2020')], 'new_St_Date': sdate2ordinal('21-Feb-2020')}
                  ]

#[item['N'] for item in DATA_DICTIONARY if item['keySet']=='bg'][0]
[item['N'] for item in DATA_DICTIONARY]

[7000000,
 11000000,
 11000000,
 60000000,
 60000000,
 80000000,
 47000000,
 126000000,
 66000000,
 80000000,
 7000000,
 150000000,
 330000000]

In [3]:
# Examples of loading different datasets
# setting up the workspace
basedir = os.getcwd()
datadir = os.path.abspath("../datasets/Bulgaria/")

# loading raw data
RawData = pd.read_csv(datadir+"/bulgaria_coronavirus.csv")
Data = pd.DataFrame({'Date' : RawData['date'].apply(sdate2ordinal),
                        'new_infected' : RawData['new_infected'],
                        'infected' : RawData['infected'],
                        'deaths' : RawData['deaths'],
                        'recovered' : RawData['recovered']
                       })

# loading optimal nodes data and managing it
OptimalNodes_raw = pd.read_csv(datadir+"/Bulgaria_optimnodes.csv")
OptimalNodes = pd.DataFrame({'Node1' : OptimalNodes_raw['fval2'].apply(sdate2ordinal), 
                             'Node2' : OptimalNodes_raw['fval4'].apply(sdate2ordinal),
                             'Fval'  : OptimalNodes_raw['fval6'],
                             'StartDate'  : OptimalNodes_raw['fval8'].apply(sdate2ordinal),
                             'EndDate'  : OptimalNodes_raw['fval10'].apply(sdate2ordinal)
                            })

# loadings learned nodes
LearnedNodes_raw = pd.DataFrame(loadmat(datadir+"/Bulgaria_History__43_45__days__Fit_R_and_I__LEARN.mat")['CC'])
LearnedNodes = pd.DataFrame({'DaysHistoryFromStart' : LearnedNodes_raw[1].astype(np.int), 
                             'betaNode' : list(flatten(LearnedNodes_raw[3])),
                             'gammaNode' : list(flatten(LearnedNodes_raw[5].array))
                            })


In [4]:
####################PLOT###############################
# Setting up the workspace
basedir = os.getcwd()
datadir = os.path.abspath("../datasets/Bulgaria/")

# Setting up global coefficients
RK = 1
Sigma = 1/11.5
SPLINE = 'pchip'
t_step = 1
SetDates_interval = [5,15,25] # SetDates1

# Setting up country (study) specific details
key = 'bg'
rawdata_file =  [item['datafile'] for item in DATA_DICTIONARY if item['keySet']==key][0]
NameState = [item['Country'] for item in DATA_DICTIONARY if item['keySet']==key][0]
NumberState = [item['valueSet'] for item in DATA_DICTIONARY if item['keySet']==key][0]
datadir = os.path.abspath("../datasets/"+str(NameState))

# loading raw data
RawData = pd.read_csv(datadir+"/" + rawdata_file)
Data = pd.DataFrame({'Date' : RawData['date'].apply(sdate2ordinal),
                        'new_infected' : RawData['new_infected'],
                        'infected' : RawData['infected'],
                        'deaths' : RawData['deaths'],
                        'recovered' : RawData['recovered']
                       })
# FileName_History
FileName_History = datadir+"/"+"Bulgaria_History__13_45__days__Fit_R_and_I__LEARN.mat"

# getting dates from raw data
Dates = Data['Date'].to_list() # Dates have already been transfered to ordinal values by managing RawData to Data
LastHistoryDate = [Dates[-1] - Dates[0] +1] #Lhistory1 
Nodes1 = [item['St-End_Date'] for item in DATA_DICTIONARY if item['keySet']==key][0]

for LHistory in LastHistoryDate:
    end_date = str(date.fromordinal(Dates[-1]))
    start = str(date.fromordinal(Nodes1[0]).strftime("%d-%b"))
    end = str(date.fromordinal(Nodes1[1]).strftime("%d-%b"))
    FileOut_name = "/"+str(NameState)+"_"+end_date+"_LRN_3rd"+"/"+str(LHistory)+"_Nodes_"+start+"-"+end+"/"
    FileOut_savedir = os.path.abspath(os.getcwd()+"/FIGURES/")
    if not os.path.exists(os.path.abspath(FileOut_savedir+FileOut_name)):
        os.makedirs(os.path.abspath(FileOut_savedir+FileOut_name))
        print("Directory ", os.path.abspath(FileOut_savedir+FileOut_name), " Created")
    else:
        print("Directory ", os.path.abspath(FileOut_savedir+FileOut_name), " already exists")
#     PLOTS_Gamma (FileOut_name, DATA_DICTIONARY, NameState, NumberState, FileName_History, Lhistory,   
#                  SetDates_interval
#                  RK, 
#                  t_step, 
#                  Sigma, 
#                  SPLINE 
#                  )
Lhistory = LastHistoryDate[0]

Directory  /home/zhana/Documents/SEIR/seir_model/source/FIGURES/Bulgaria_2020-04-21_LRN_3rd/45_Nodes_14-Mar-09-Apr  Created


In [5]:
#def     PLOTS_Gamma (FileOut_name, DATA_DICTIONARY, NameState, NumberState, FileName_History, FileOut_savedir
#                  Lhistory,   
#                  SetDates_interval
#                  RK, 
#                  t_step, 
#                  Sigma, 
#                  SPLINE 
#                  ):

DIVERGENT = [] # collect all cases that did not converge

# load RawData and pull relevant information from it
rawdata_file =  [item['datafile'] for item in DATA_DICTIONARY if item['valueSet']==NumberState][0]
RawData = pd.read_csv(datadir+"/" + rawdata_file)
Data = pd.DataFrame({'Date' : RawData['date'].apply(sdate2ordinal),
                        'new_infected' : RawData['new_infected'],
                        'infected' : RawData['infected'],
                        'deaths' : RawData['deaths'],
                        'recovered' : RawData['recovered']
                       })

Dates = Data['Date'].to_list() # Dates have already been transfered to ordinal values by managing RawData to Data
Nodes1 = [item['St-End_Date'] for item in DATA_DICTIONARY if item['keySet']==key][0]
N = [item['N'] for item in DATA_DICTIONARY if item['keySet']==key][0]

# Load FileName_History: learned nodes
LearnedNodes_raw = pd.DataFrame(loadmat(FileName_History)['CC'])
LearnedNodes = pd.DataFrame({'DaysHistoryFromStart' : LearnedNodes_raw[1].astype(np.int), 
                             'betaNode' : list(flatten(LearnedNodes_raw[3])),
                             'gammaNode' : list(flatten(LearnedNodes_raw[5].array))
                            })

# get DaysHistoryFromStart - betaNode - gammaNode - Start and EndDates
days_start_index = [i for i in range(LearnedNodes.shape[0]) if [x-Lhistory for x in LearnedNodes['DaysHistoryFromStart'].to_list()][i]==0][0]
DaysHistoryFromStart = LearnedNodes['DaysHistoryFromStart'][days_start_index]
betaNode = LearnedNodes['betaNode'][days_start_index]
gammaNode = LearnedNodes['gammaNode'][days_start_index]
StartDate = Dates[0]
EndDate0 = Dates[0] + DaysHistoryFromStart -1
Nodes = list(flatten([[StartDate], Nodes1, [EndDate0]]) )

# get relevant raw data
I_data = Data['new_infected'].to_list()
R_data = Data['recovered'].to_list()
Deaths = Data['deaths'].to_list()
I0 = I_data[0]
E0 = 0
R0 = R_data[0]+Deaths[0]
S0 = N-I0-E0-R0

# TODO: add DaysToPredict as an extra variable for the function
DaysToPredict = 55
EndDate1 = EndDate0+DaysToPredict

SetDates1 = [x+EndDate0 for x in SetDates_interval] #  SetDates_interval is [ 5  15  25 ] --> SetDates0

for Tbreak in SetDates1:
    for coefB in np.round(np.arange(0.2, 1.4+0.0001, 0.2), decimals=1):
        for coefG in np.round(np.arange(0.6, 1.8+0.0001, 0.2),  decimals=1):
            for coefBrx in np.round(np.arange(0.2, 1+0.0001, 0.4), decimals=1):
                for coefGrx in np.round(np.arange(0.2, 1+0.0001, 0.4), decimals=1):
                    print(Tbreak,coefB,coefG,coefBrx,coefGrx)
                    # derive coefficients
                    # TODO : get grids as function variables
                    grid1 = [0.2, 1, 1.4];
                    grid2 = [1.4, 1, 0.2];
                    coefBamp = sci.CubicSpline(grid1, grid2, axis=0, bc_type='not-a-knot')(coefB).tolist() # Matlab's csapi uses the not-a-knot end condition
                    # TODO : pull beta and gamma priors as function variables
                    ctrBeta0 = 0.8;
                    if betaNode[-1]==0:
                        ctrBeta =  coefBamp*0.5*(betaNode[-1]+betaNode[-2])
                    else:
                        ctrBeta =  coefBamp*betaNode[-1]
                    ctrBeta = [ctrBeta,(ctrBeta0+coefBrx)*ctrBeta];
                    ctrGamma0 = 1.2;
                    ctrGamma = coefG*gammaNode[-1];
                    ctrGamma = [ctrGamma,(ctrGamma0-coefGrx)*ctrGamma] ; 
                    # predict SEIR values    
                    [S,E,I,R,gamma] = seir_spline_predict(StartDate, EndDate0, EndDate1, Tbreak, ctrBeta, ctrGamma, Nodes, betaNode, gammaNode, N, S0, E0, I0, R0, SPLINE, Sigma, RK)
                    #seir_plot(S,E,I,R,TimeSeries,title_add=' for Simulated Data', fig_save=1, fig_text='_Simulation')
                    # plot prediction
                    I_predicted = [0 if x < 0 else x for x in I]
                    TimeSeries = range(StartDate,EndDate1+1)
                    # decide on whether a plot shall be produced or if model is diverging
                    # TODO: this is a bad strategy - the gamma plot is currently plotting before deciding this whether to output plot
                    fig_save = 0
                    if abs(I[-1]) < S0: 
                        fig_save = 1
                    #########################################################
                    divergent_number = gamma_plot(I_data,I_predicted,TimeSeries, DaysHistoryFromStart, StartDate, EndDate0, EndDate1, Nodes, Tbreak, SetDates1, coefB, coefBrx, coefG, coefGrx, NameState, FileOut_name, FileOut_savedir, fig_save)
                    DIVERGENT.append(divergent_number)
#TODO: save divergent results

737541 0.2 0.6 0.2 0.2
737541 0.2 0.6 0.2 0.6
737541 0.2 0.6 0.2 1.0
737541 0.2 0.6 0.6 0.2
737541 0.2 0.6 0.6 0.6
737541 0.2 0.6 0.6 1.0
737541 0.2 0.6 1.0 0.2
737541 0.2 0.6 1.0 0.6
737541 0.2 0.6 1.0 1.0
737541 0.2 0.8 0.2 0.2
737541 0.2 0.8 0.2 0.6
737541 0.2 0.8 0.2 1.0
737541 0.2 0.8 0.6 0.2
737541 0.2 0.8 0.6 0.6
737541 0.2 0.8 0.6 1.0
737541 0.2 0.8 1.0 0.2
737541 0.2 0.8 1.0 0.6
737541 0.2 0.8 1.0 1.0
737541 0.2 1.0 0.2 0.2
737541 0.2 1.0 0.2 0.6
737541 0.2 1.0 0.2 1.0
737541 0.2 1.0 0.6 0.2
737541 0.2 1.0 0.6 0.6
737541 0.2 1.0 0.6 1.0
737541 0.2 1.0 1.0 0.2
737541 0.2 1.0 1.0 0.6
737541 0.2 1.0 1.0 1.0
737541 0.2 1.2 0.2 0.2
737541 0.2 1.2 0.2 0.6
737541 0.2 1.2 0.2 1.0
737541 0.2 1.2 0.6 0.2
737541 0.2 1.2 0.6 0.6
737541 0.2 1.2 0.6 1.0
737541 0.2 1.2 1.0 0.2
737541 0.2 1.2 1.0 0.6
737541 0.2 1.2 1.0 1.0
737541 0.2 1.4 0.2 0.2
737541 0.2 1.4 0.2 0.6
737541 0.2 1.4 0.2 1.0
737541 0.2 1.4 0.6 0.2
737541 0.2 1.4 0.6 0.6
737541 0.2 1.4 0.6 1.0
737541 0.2 1.4 1.0 0.2
737541 0.2 

/home/zhana/Documents/SEIR/seir_model/source/utils.py:99: RuntimeWarning: overflow encountered in double_scalars
  S1 = -0.5*(beta[t]+beta[t+1])*S2*I2/N
/home/zhana/Documents/SEIR/seir_model/source/utils.py:100: RuntimeWarning: overflow encountered in double_scalars
  E1 = 0.5*(beta[t]+beta[t+1])*S2*I2/N - sigma*E2
/home/zhana/Documents/SEIR/seir_model/source/utils.py:113: RuntimeWarning: invalid value encountered in double_scalars
  E1 = beta[t+1]*S2*I2/N - sigma*E2
/home/zhana/Documents/SEIR/seir_model/source/utils.py:189: RuntimeWarning: invalid value encountered in double_scalars
  SEIRTotal =  [x + (1/6)*(y+2*z+2*u+v) for x,y,z,u,v in zip([S[-1],E[-1],I[-1],R[-1]],SEIR0,SEIR11,SEIR12,SEIR2)]


737541 0.2 1.6 0.2 1.0
737541 0.2 1.6 0.6 0.2
737541 0.2 1.6 0.6 0.6


/home/zhana/Documents/SEIR/seir_model/source/utils.py:112: RuntimeWarning: overflow encountered in double_scalars
  S1 = -beta[t+1]*S2*I2/N
/home/zhana/Documents/SEIR/seir_model/source/utils.py:113: RuntimeWarning: overflow encountered in double_scalars
  E1 = beta[t+1]*S2*I2/N - sigma*E2
/home/zhana/Documents/SEIR/seir_model/source/utils.py:186: RuntimeWarning: invalid value encountered in double_scalars
  SEIR11 = F1(time_ix,[x + y/2 for x,y in zip([S[-1],E[-1],I[-1],R[-1]],SEIR0)],N,beta,gamma,Sigma)


737541 0.2 1.6 0.6 1.0
737541 0.2 1.6 1.0 0.2
737541 0.2 1.6 1.0 0.6


/home/zhana/Documents/SEIR/seir_model/source/utils.py:87: RuntimeWarning: invalid value encountered in double_scalars
  E1 = beta[t]*S2*I2/N - sigma*E2
/home/zhana/Documents/SEIR/seir_model/source/utils.py:86: RuntimeWarning: overflow encountered in double_scalars
  S1 = -beta[t]*S2*I2/N
/home/zhana/Documents/SEIR/seir_model/source/utils.py:87: RuntimeWarning: overflow encountered in double_scalars
  E1 = beta[t]*S2*I2/N - sigma*E2
/home/zhana/Documents/SEIR/seir_model/source/utils.py:100: RuntimeWarning: invalid value encountered in double_scalars
  E1 = 0.5*(beta[t]+beta[t+1])*S2*I2/N - sigma*E2


737541 0.2 1.6 1.0 1.0
737541 0.2 1.8 0.2 0.2
737541 0.2 1.8 0.2 0.6
737541 0.2 1.8 0.2 1.0
737541 0.2 1.8 0.6 0.2
737541 0.2 1.8 0.6 0.6
737541 0.2 1.8 0.6 1.0
737541 0.2 1.8 1.0 0.2
737541 0.2 1.8 1.0 0.6
737541 0.2 1.8 1.0 1.0
737541 0.4 0.6 0.2 0.2
737541 0.4 0.6 0.2 0.6
737541 0.4 0.6 0.2 1.0
737541 0.4 0.6 0.6 0.2
737541 0.4 0.6 0.6 0.6
737541 0.4 0.6 0.6 1.0
737541 0.4 0.6 1.0 0.2
737541 0.4 0.6 1.0 0.6
737541 0.4 0.6 1.0 1.0
737541 0.4 0.8 0.2 0.2
737541 0.4 0.8 0.2 0.6
737541 0.4 0.8 0.2 1.0
737541 0.4 0.8 0.6 0.2
737541 0.4 0.8 0.6 0.6
737541 0.4 0.8 0.6 1.0
737541 0.4 0.8 1.0 0.2
737541 0.4 0.8 1.0 0.6
737541 0.4 0.8 1.0 1.0
737541 0.4 1.0 0.2 0.2
737541 0.4 1.0 0.2 0.6
737541 0.4 1.0 0.2 1.0
737541 0.4 1.0 0.6 0.2
737541 0.4 1.0 0.6 0.6
737541 0.4 1.0 0.6 1.0
737541 0.4 1.0 1.0 0.2
737541 0.4 1.0 1.0 0.6
737541 0.4 1.0 1.0 1.0
737541 0.4 1.2 0.2 0.2
737541 0.4 1.2 0.2 0.6
737541 0.4 1.2 0.2 1.0
737541 0.4 1.2 0.6 0.2
737541 0.4 1.2 0.6 0.6
737541 0.4 1.2 0.6 1.0
737541 0.4 

KeyboardInterrupt: 

<Figure size 1440x864 with 0 Axes>